In [1]:
!pip install spatial

  Preparing metadata (setup.py) ... done
  Created wheel for spatial: filename=spatial-0.2.0-py3-none-any.whl size=8484 sha256=cc1f7757e99335df87d07e9e77aaa99a95673be3f7db26f1c42aa4e25d36854b
  Stored in directory: /root/.cache/pip/wheels/b3/a9/45/5776d9357e179ab80f60dfbf7d002d095f1a7c5d8f86104bd3
Successfully built spatial


In [2]:
!pip install scipy

In [3]:
!pip install sktime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 kB 12.1 MB/s eta 0:00:00


In [4]:
!pip install dtaidistance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.5 MB/s eta 0:00:00


In [ ]:
# The idea of this notebook is to use the pretrained model ResNet18 and to measure similarity

In [5]:
# Import the necessary packages, modules, functions, and methods
import torch
import torchvision
from torchvision import models
import spatial
import scipy.spatial.distance
from torchsummary import summary
from keras.models import Model
import sktime
from sktime.distances import euclidean_distance
from dtaidistance import dtw
import dtaidistance
from sklearn import preprocessing

In [6]:
# mount my google drive so I can import files from there
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Load the pre-trained ResNet18 model
resnet = models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 56.9MB/s]


In [8]:
#show the layers of ResNet18
summary(resnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [9]:
# Access the desired layer ("avgpool") in PyTorch
avg_pool_layer = resnet.avgpool  # Access the "avgpool" layer directly

In [10]:
# Define a new model that outputs the features from the desired layer
class FeatureExtractor(torch.nn.Module):  # Create a new class for the feature extractor
    def __init__(self, base_model, layer):  # Initialize the class with the base model and layer
        super(FeatureExtractor, self).__init__()  # Call the parent class constructor
        self.base_model = base_model  # Store the base model
        self.layer = layer  # Store the desired layer

    def forward(self, x):  # Forward pass
        for name, module in self.base_model._modules.items():  # Iterate through the layers of the base model
            x = module(x)  # Apply the layer
            if name == self.layer:  # Check if the desired layer is reached
                break  # Exit the loop
        return x  # Return the output of the desired layer

In [11]:
# not all layers of the model are used. We extract the vector from layer "AdaptiveAvgPool2d” as an extracted feature of an image
model = FeatureExtractor(resnet, 'avgpool')  # Create the feature extractor

In [ ]:
#below we define three functions that we use to calcualte similarity

In [12]:
# function for calculating cosine similarity
def cosine_similarity(vector1, vector2):
  vector1 = vector1.flatten() # Flatten the tensor to 1D
  vector2 = vector2.flatten() # Flatten the tensor to 1D
  return 1 - scipy.spatial.distance.cosine(vector1.detach().numpy(), vector2.detach().numpy()) # Detach the tensors from the computation graph and convert them to NumPy arrays

In [13]:
# function for calculating euclidean similarity
def euclidean_similarity(x, y):
    x = x.flatten() # Flatten the tensor to 1D
    y = y.flatten() # Flatten the tensor to 1D
    return 1 / (1 + euclidean_distance(x.detach().numpy(), y.detach().numpy())) # Convert tensors to NumPy arrays before calculating the distance

In [14]:
# function for calculating dynamic time warping similarity of vectors
def dtw_similarity(time_series_a,time_series_b):
  # dtw.warping_paths returns the DTW distance and the DTW cost matrix
  distance, paths = dtw.warping_paths(time_series_a.flatten(), time_series_b.flatten(), use_c=False)
  # Compute the optimal path from the DTW cost matrix
  best_path = dtw.best_path(paths)
  return 1 - (distance / len(best_path)) # Use dtaidistance to caluate the dynamic time warping similarity of the vectors

# function for scaling data in an array
def normalize_series(data):
  # collapse the array into one dimension
  data_array = data.flatten()
  # Detach the tensor from the computation graph and convert it to a NumPy array
  data_array = data_array.detach().numpy()
  # Normalize the data using MinMaxScaler
  scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
  # Fit the scaler on the data and transform it
  normalized_array = scaler.fit_transform(data_array.reshape(-1, 1))
  return normalized_array # return the normalized array

In [ ]:
#We check the similarity between similar songs involved in famous court cases using cosine similarity, euclidean similarity, and dynamic time warping similarity
#In each case the closer the score is to 0 the more dissimilar the songs are while the closer the score is to 1 the more similiar the songs are

In [25]:
# Vanilla Ice's Ice Ice Baby vs. Under Pressure by Queen and David Bowie
copy1 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IceIceBaby_full.jpg").float() # Convert image to float
org1 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/UnderPressure_full.jpg").float() # Convert image to float
# Add a batch dimension to the images
org1 = org1.unsqueeze(0) # Add a batch dimension (dim=0)
copy1 = copy1.unsqueeze(0) # Add a batch dimension (dim=0)
feat1 = model(org1)
feat2 = model(copy1)
cosine_similarity(feat1, feat2), euclidean_similarity(feat1, feat2), dtw_similarity(normalize_series(feat1),normalize_series(feat2))

(0.9992821540258834, 0.5419283239776682, 0.9981487904831516)

In [26]:
# the Verve's Bitter Sweet Symphony vs. the Rolling Stone's The Last Time
org2 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/BittersweetSymphony_full.jpg").float()
copy2 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheLastTime_full.jpg").float()
# Add a batch dimension to the images
org2 = org2.unsqueeze(0) # Add a batch dimension (dim=0)
copy2 = copy2.unsqueeze(0) # Add a batch dimension (dim=0)
aeat1 = model(org2)
aeat2 = model(copy2)
cosine_similarity(aeat1, aeat2), euclidean_similarity(aeat1, aeat2), dtw_similarity(normalize_series(aeat1),normalize_series(aeat2))

(0.9990979880138519, 0.5240111639617179, 0.9978767695798398)

In [27]:
# Robin Thicke featuring rapper T.I. and singer Pharrell Williams's Blurred Lines vs. Marvin Gaye's Gotta Give It Up
org3 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/GottaGiveItUp_full.jpg").float()
copy3 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/blurredlines_full.jpg").float()
# Add a batch dimension to the images
org3 = org3.unsqueeze(0) # Add a batch dimension (dim=0)
copy3 = copy3.unsqueeze(0) # Add a batch dimension (dim=0)
beat1 = model(org3)
beat2 = model(copy3)
cosine_similarity(beat1, beat2), euclidean_similarity(beat1, beat2), dtw_similarity(normalize_series(beat1),normalize_series(beat2))

(0.9993273314838035, 0.5716701051661558, 0.9975071974286073)

In [28]:
# Killing Joke‘s Eighties vs. Nirvana's Come as You are
copy4 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ComeAsYouAre_full.jpg").float()
org4 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheKillingJoke_full.jpg").float()
# Add a batch dimension to the images
org4 = org4.unsqueeze(0) # Add a batch dimension (dim=0)
copy4 = copy4.unsqueeze(0) # Add a batch dimension (dim=0)
ceat1 = model(org4)
ceat2 = model(copy4)
cosine_similarity(ceat1, ceat2), euclidean_similarity(ceat1, ceat2), dtw_similarity(normalize_series(ceat1),normalize_series(ceat2))

(0.9991937772162273, 0.5464573885940459, 0.9976911216663776)

In [29]:
# Chuck Berry's You Can't Catch Me vs. The Beatles Come Together
org5 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouCantCatchMe_full.jpg").float()
copy5 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ComeTogether_full.jpg").float()
# Add a batch dimension to the images
org5 = org5.unsqueeze(0) # Add a batch dimension (dim=0)
copy5 = copy5.unsqueeze(0) # Add a batch dimension (dim=0)
deat1 = model(org5)
deat2 = model(copy5)
cosine_similarity(deat1, deat2), euclidean_similarity(deat1, deat2), dtw_similarity(normalize_series(deat1),normalize_series(deat2))

(0.999188850071174, 0.5334815611486301, 0.9980070342131827)

In [30]:
# Radiohead's Creep vs. Lana Del Rey's Get Free
org6 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Creep_full.jpg").float()
copy6 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/GetFree_full.jpg").float()\
# Add a batch dimension to the images
org6 = org6.unsqueeze(0) # Add a batch dimension (dim=0)
copy6 = copy6.unsqueeze(0) # Add a batch dimension (dim=0)
eeat1 = model(org6)
eeat2 = model(copy6)
cosine_similarity(eeat1, eeat2), euclidean_similarity(eeat1, eeat2), dtw_similarity(normalize_series(eeat1),normalize_series(eeat2))

(0.9990838983298976, 0.44889875057657913, 0.9978866979742061)

In [31]:
# Radiohead's Creep vs. The Hollies The Air That I Breathe
org7 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheAirThatIBreathe_full.jpg").float()
copy7 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Creep_full.jpg").float()
# Add a batch dimension to the images
org7 = org7.unsqueeze(0) # Add a batch dimension (dim=0)
copy7 = copy7.unsqueeze(0) # Add a batch dimension (dim=0)
geat1 = model(org7)
geat2 = model(copy7)
cosine_similarity(geat1, geat2), euclidean_similarity(geat1, geat2), dtw_similarity(normalize_series(geat1),normalize_series(geat2))

(0.9992452623122817, 0.5315498470419161, 0.997770859233575)

In [32]:
# George Harrison's My Sweet Lord vs. the Chiffons’ He’s So Fine
org8 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Hessofine_full.jpg").float()
copy8 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/MySweetLord_full.jpg").float()
# Add a batch dimension to the images
org8 = org8.unsqueeze(0) # Add a batch dimension (dim=0)
copy8 = copy8.unsqueeze(0) # Add a batch dimension (dim=0)
heat1 = model(org8)
heat2 = model(copy8)
cosine_similarity(heat1, heat2), euclidean_similarity(heat1, heat2), dtw_similarity(normalize_series(heat1),normalize_series(heat2))

(0.999371634081829, 0.5747763634745483, 0.9981076595072349)

In [33]:
# Katy Perry featuring Juicy J.'s Dark Horse vs. Flame's Joyful Noise
org9 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/JoyfulNoise_full.jpg").float()
copy9 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/DarkHorse_full.jpg").float()
# Add a batch dimension to the images
org9 = org9.unsqueeze(0) # Add a batch dimension (dim=0)
copy9 = copy9.unsqueeze(0) # Add a batch dimension (dim=0)
jeat1 = model(org9)
jeat2 = model(copy9)
cosine_similarity(jeat1, jeat2), euclidean_similarity(jeat1, jeat2), dtw_similarity(normalize_series(jeat1),normalize_series(jeat2))

(0.9990798896625854, 0.5278104732133297, 0.9978215751382793)

In [34]:
# Rod Stewart's Do You Think I'm Sexy vs. Jorge Ben jor's Taj Mahal
org10 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TajMahal_full.jpg").float()
copy10 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/DoYaThinkImSexy_full.jpg").float()
# Add a batch dimension to the images
org10 = org10.unsqueeze(0) # Add a batch dimension (dim=0)
copy10 = copy10.unsqueeze(0) # Add a batch dimension (dim=0)
keat1 = model(org10)
keat2 = model(copy10)
cosine_similarity(keat1, keat2), euclidean_similarity(keat1, keat2), dtw_similarity(normalize_series(keat1),normalize_series(keat2))

(0.9990406151430725, 0.5240343748060811, 0.9968844204270496)

In [35]:
# Ray Parker Jr.'s Ghostbusters vs. Huey Lewis's I Want a New Drug
copy11 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Ghostbusters_full.jpg").float()
org11 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IWantaNewDrug_full.jpg").float()
# Add a batch dimension to the images
org11 = org11.unsqueeze(0) # Add a batch dimension (dim=0)
copy11 = copy11.unsqueeze(0) # Add a batch dimension (dim=0)
leat1 = model(org11)
leat2 = model(copy11)
cosine_similarity(leat1, leat2), euclidean_similarity(leat1, leat2), dtw_similarity(normalize_series(leat1),normalize_series(leat2))

(0.9991662877509471, 0.5321089895895997, 0.9977565731951543)

In [36]:
# Olivia Rodrigo's Good 4 U vs. Paramore's Misery Business
org12 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/MiseryBusiness_full.jpg").float()
copy12 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Good4u_full.jpg").float()
# Add a batch dimension to the images
org12 = org12.unsqueeze(0) # Add a batch dimension (dim=0)
copy12 = copy12.unsqueeze(0) # Add a batch dimension (dim=0)
meat1 = model(org12)
meat2 = model(copy12)
cosine_similarity(meat1, meat2), euclidean_similarity(meat1, meat2), dtw_similarity(normalize_series(meat1),normalize_series(meat2))

(0.9993031814965958, 0.5586367857317895, 0.9982238979881509)

In [37]:
#Oasis's Shakermaker vs. The New Seekers's I’d Like To Teach The World To Sing
org13 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/I'dLiketoTeaktheworldtosing_full.jpg").float()
copy13 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Shakermaker_full.jpg").float()
# Add a batch dimension to the images
org13 = org13.unsqueeze(0) # Add a batch dimension (dim=0)
copy13 = copy13.unsqueeze(0) # Add a batch dimension (dim=0)
neat1 = model(org13)
neat2 = model(copy13)
cosine_similarity(neat1, neat2), euclidean_similarity(neat1, neat2), dtw_similarity(normalize_series(neat1),normalize_series(neat2))

(0.9991825707921904, 0.5458719048320047, 0.9982912680342112)

In [38]:
# Marvin Gaye's Let's Get It On vs. Ed Sheeran's Thinking Out Loud
org14 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/LetsGetItOn_full.jpg").float()
copy14 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ThinkingOutLoud_full.jpg").float()
# Add a batch dimension to the images
org14 = org14.unsqueeze(0) # Add a batch dimension (dim=0)
copy14 = copy14.unsqueeze(0) # Add a batch dimension (dim=0)
peat1 = model(org14)
peat2 = model(copy14)
cosine_similarity(peat1, peat2), euclidean_similarity(peat1, peat2), dtw_similarity(normalize_series(peat1),normalize_series(peat2))

(0.9993253219287899, 0.5577236039584025, 0.9978212632491323)

In [39]:
# Led Zeppelin's Stairway to Heaven vs. Spirit's Taurus
org15 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Taurus_full.jpg").float()
copy15 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/StairwaytoHeaven_full.jpg").float()
# Add a batch dimension to the images
org15 = org15.unsqueeze(0) # Add a batch dimension (dim=0)
copy15 = copy15.unsqueeze(0) # Add a batch dimension (dim=0)
qeat1 = model(org15)
qeat2 = model(copy15)
cosine_similarity(qeat1, qeat2), euclidean_similarity(qeat1, qeat2), dtw_similarity(normalize_series(qeat1),normalize_series(qeat2))

(0.9992473359444322, 0.5595595055316047, 0.9978632826669471)

In [40]:
# Sam Smith's Stay with Me vs. Tom Petty's I won't Back Down
org16 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IWontBackDown_full.jpg").float()
copy16 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/StayWithMe_full.jpg").float()
# Add a batch dimension to the images
org16 = org16.unsqueeze(0) # Add a batch dimension (dim=0)
copy16 = copy16.unsqueeze(0) # Add a batch dimension (dim=0)
reat1 = model(org16)
reat2 = model(copy16)
cosine_similarity(reat1, reat2), euclidean_similarity(reat1, reat2), dtw_similarity(normalize_series(reat1),normalize_series(reat2))

(0.9989713569254658, 0.5197388376830828, 0.9971416063899966)

In [41]:
# Beach Boy's Surfin USA vs. Chuck Berry's Sweet Little Sixteen
org17 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/SweetLittleSixteen_full.jpg").float()
copy17 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/SurfinUSA_full.jpg").float()
# Add a batch dimension to the images
org17 = org17.unsqueeze(0) # Add a batch dimension (dim=0)
copy17 = copy17.unsqueeze(0) # Add a batch dimension (dim=0)
seat1 = model(org17)
seat2 = model(copy17)
cosine_similarity(seat1, seat2), euclidean_similarity(seat1, seat2), dtw_similarity(normalize_series(seat1),normalize_series(seat2))

(0.9988502648547276, 0.5036628665686765, 0.9977504618510985)

In [42]:
# De La Soul's Transmitting Live From Mars vs. The Turtles's You Showed Me
org18 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouShowedMe_full.jpg").float()
copy18 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TransmittingLivefromMars_full.jpg").float()
# Add a batch dimension to the images
org18 = org18.unsqueeze(0) # Add a batch dimension (dim=0)
copy18 = copy18.unsqueeze(0) # Add a batch dimension (dim=0)
teat1 = model(org18)
teat2 = model(copy18)
cosine_similarity(teat1, teat2), euclidean_similarity(teat1, teat2), dtw_similarity(normalize_series(teat1),normalize_series(teat2))

(0.9992765945592654, 0.5518787031394029, 0.9976377874731227)

In [15]:
# Coldplay's Viva La Vida vs. Joe Satriani's If I Could Fly
org19 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IfICouldFly_full.jpg").float()
copy19 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/VivaLaVida_full.jpg").float()
# Add a batch dimension to the images
org19 = org19.unsqueeze(0) # Add a batch dimension (dim=0)
copy19 = copy19.unsqueeze(0) # Add a batch dimension (dim=0)
veat1 = model(org19)
veat2 = model(copy19)
cosine_similarity(veat1, veat2), euclidean_similarity(veat1, veat2), dtw_similarity(normalize_series(veat1),normalize_series(veat2))

(0.9990314195652162, 0.5192915077397642, 0.997351265633777)

In [16]:
# Led Zeppelin's Whole Lotta Love vs. Muddy Waters's You Need Love
org20 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouNeedLove_full.jpg").float()
copy20 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/WholeLottaLove_full.jpg").float()
# Add a batch dimension to the images
org20 = org20.unsqueeze(0) # Add a batch dimension (dim=0)
copy20 = copy20.unsqueeze(0) # Add a batch dimension (dim=0)
weat1 = model(org20)
weat2 = model(copy20)
cosine_similarity(weat1, weat2), euclidean_similarity(weat1, weat2), dtw_similarity(normalize_series(weat1),normalize_series(weat2))

(0.9990824276615689, 0.518089109690695, 0.9981876487728747)

In [ ]:
#We check the similarity between songs that are very dissimilar to the human ear

In [17]:
#J.S. Bach - Toccata and Fugue in D minor BWV 565 vs. Master Of Puppets - Metallica
org21 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/J.S. Bach - Toccata and Fugue in D minor BWV 565.jpg").float()
copy21 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Master Of Puppets - Metallica.jpg").float()
org21 = org21.unsqueeze(0) # Add a batch dimension (dim=0)
copy21 = copy21.unsqueeze(0) # Add a batch dimension (dim=0)
aaeat1 = model(org21)
aaeat2 = model(copy21)
cosine_similarity(aaeat1, aaeat2), euclidean_similarity(aaeat1, aaeat2), dtw_similarity(normalize_series(aaeat1),normalize_series(aaeat2))

(0.9990676566042378, 0.5321980597140543, 0.9979426307230087)

In [18]:
# Michael Jackson - Billie Jean vs. Alice Cooper - Schools Out
org22 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Michael Jackson - Billie Jean.jpg").float()
copy22 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Alice Cooper - Schools Out.jpg").float()
org22 = org22.unsqueeze(0) # Add a batch dimension (dim=0)
copy22 = copy22.unsqueeze(0) # Add a batch dimension (dim=0)
bbeat1 = model(org22)
bbeat2 = model(copy22)
cosine_similarity(bbeat1, bbeat2), euclidean_similarity(bbeat1, bbeat2), dtw_similarity(normalize_series(bbeat1),normalize_series(bbeat2))

(0.9992277435852752, 0.5283894583865238, 0.9978746552923841)

In [19]:
# Yakety Sax - Boots Randolph vs. Vanessa Carlton - A Thousand miles
org23 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Yakety Sax - Boots Randolph.jpg").float()
copy23 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Vanessa Carlton - A Thousand Miles.jpg").float()
org23 = org23.unsqueeze(0) # Add a batch dimension (dim=0)
copy23 = copy23.unsqueeze(0) # Add a batch dimension (dim=0)
cceat1 = model(org23)
cceat2 = model(copy23)
cosine_similarity(cceat1, cceat2), euclidean_similarity(cceat1, cceat2), dtw_similarity(normalize_series(cceat1),normalize_series(cceat2))

(0.9992825192391533, 0.5541912589330013, 0.9980206792452436)

In [20]:
# AC⧸DC - Back In Black vs. 4'33'' - John Cage
org24 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/AC⧸DC - Back In Black.jpg").float()
copy24 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/4'33'' - John Cage.jpg").float()
org24 = org24.unsqueeze(0) # Add a batch dimension (dim=0)
copy24 = copy24.unsqueeze(0) # Add a batch dimension (dim=0)
ddeat1 = model(org24)
ddeat2 = model(copy24)
cosine_similarity(ddeat1, ddeat2), euclidean_similarity(ddeat1, ddeat2), dtw_similarity(normalize_series(ddeat1),normalize_series(ddeat2))

(0.9991314775018219, 0.5120526517096, 0.9979148729035836)

In [21]:
# Tequila - The Champs  vs. Chopin - Nocturne in E Flat Major (Op. 9 No. 2)
org25 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Tequila - The Champs.jpg").float()
copy25 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Chopin - Nocturne in E Flat Major (Op. 9 No. 2).jpg").float()
org25 = org25.unsqueeze(0) # Add a batch dimension (dim=0)
copy25 = copy25.unsqueeze(0) # Add a batch dimension (dim=0)
eeeat1 = model(org25)
eeeat2 = model(copy25)
cosine_similarity(eeeat1, eeeat2), euclidean_similarity(eeeat1, eeeat2), dtw_similarity(normalize_series(eeeat1),normalize_series(eeeat2))

(0.9989729324208803, 0.5198437389524883, 0.9973985296314184)

In [22]:
# Aretha Franklin - Respect vs. Love Theme From Romeo & Juliet - Pyotr Ilyich Tchaikovsky
org26 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Aretha Franklin - Respect.jpg").float()
copy26 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Love Theme From Romeo & Juliet - Pyotr Ilyich Tchaikovsky.jpg").float()
org26 = org26.unsqueeze(0) # Add a batch dimension (dim=0)
copy26 = copy26.unsqueeze(0) # Add a batch dimension (dim=0)
ffeat1 = model(org26)
ffeat2 = model(copy26)
cosine_similarity(ffeat1, ffeat2), euclidean_similarity(ffeat1, ffeat2), dtw_similarity(normalize_series(ffeat1),normalize_series(ffeat2))

(0.9993310921390113, 0.5506802535952106, 0.9978887523238066)

In [23]:
# Nirvana - Smells Like Teen Spirit vs. Ave Maria - J.S. Bach and Charles Gounod
org27 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Nirvana - Smells Like Teen Spirit.jpg").float()
copy27 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Ave Maria - J.S. Bach and Charles Gounod.jpg").float()
org27 = org27.unsqueeze(0) # Add a batch dimension (dim=0)
copy27 = copy27.unsqueeze(0) # Add a batch dimension (dim=0)
ggeat1 = model(org27)
ggeat2 = model(copy27)
cosine_similarity(ggeat1, ggeat2), euclidean_similarity(ggeat1, ggeat2), dtw_similarity(normalize_series(ggeat1),normalize_series(ggeat2))

(0.9991490202406585, 0.5007718287635903, 0.997813955625132)

In [24]:
# David Bowie - Heroes vs. OutKast - Hey Ya
org28 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/David Bowie - Heroes.jpg").float()
copy28 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/OutKast - Hey Ya.jpg").float()
org28 = org28.unsqueeze(0) # Add a batch dimension (dim=0)
copy28 = copy28.unsqueeze(0) # Add a batch dimension (dim=0)
hheat1 = model(org28)
hheat2 = model(copy28)
cosine_similarity(hheat1, hheat2), euclidean_similarity(hheat1, hheat2), dtw_similarity(normalize_series(hheat1),normalize_series(hheat2))

(0.9990322711632141, 0.5284367980462462, 0.9978800901118343)

In [25]:
# The Dave Brubeck Quartet - Take Five vs. Public Enemy - Fight the Power
org29 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/The Dave Brubeck Quartet - Take Five.jpg").float()
copy29 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Public Enemy - Fight the Power.jpg").float()
org29 = org29.unsqueeze(0) # Add a batch dimension (dim=0)
copy29 = copy29.unsqueeze(0) # Add a batch dimension (dim=0)
jjeat1 = model(org29)
jjeat2 = model(copy29)
cosine_similarity(jjeat1, jjeat2), euclidean_similarity(jjeat1, jjeat2), dtw_similarity(normalize_series(jjeat1),normalize_series(jjeat2))

(0.9991394953851029, 0.5379145418076379, 0.9974208606335849)

In [26]:
#Grieg - Peer Gynt Suite No. 1, Op. 46 I. Morning Mood vs. Pour Some Sugar On Me - Def Leppard
org30 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Grieg - Peer Gynt Suite No. 1, Op. 46 I. Morning Mood.jpg").float()
copy30 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Pour Some Sugar On Me - Def Leppard.jpg").float()
org30 = org30.unsqueeze(0) # Add a batch dimension (dim=0)
copy30 = copy30.unsqueeze(0) # Add a batch dimension (dim=0)
kkeat1 = model(org30)
kkeat2 = model(copy30)
cosine_similarity(kkeat1, kkeat2), euclidean_similarity(kkeat1, kkeat2), dtw_similarity(normalize_series(kkeat1),normalize_series(kkeat2))

(0.9990664059616721, 0.5298493489543532, 0.9978850177993422)

In [ ]:
#The scores are extremely high (close to 1) for all song pairs when evaluated by cosine and DTW similarity.
#The scores are all about midway between 0 and 1 for all song pairs when evaluated by euclidean similarity.
#This model as-is is not a terribly good representation of similarity between songs.